In [1]:
import pandas as pd
import mysql.connector

In [103]:
host = "feenix-mariadb.swin.edu.au"
user = "s104489467"
password = "101100"
database = "s104489467_db"
connection = mysql.connector.connect(host=host,user=user,password=password,database=database)
cursor = connection.cursor()

In [3]:
sql_query = "select * from public_sentiment"

In [37]:
df = pd.read_sql(sql_query, connection)

C:\Users\HP Envy\AppData\Local\Temp\ipykernel_2844\1690567701.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


In [38]:
df['key_word'] = df['key_word'].str.strip() \
    .str.replace('-', '', regex=False) 

In [39]:
df['key_word'] = df['key_word'].str.replace(r'\\n', ', ', regex=True)

In [40]:
df['key_word'] = df['key_word'].str.replace(r'\*', ', ', regex=True) 
df['key_word'] = df['key_word'].str.replace(r'\d+', ', ', regex=True) 

In [44]:
df['key_word'] =df['key_word'].str.replace(r', , ',', ')

In [46]:
df['key_word'] =df['key_word'].str.replace(r' , ',', ')

In [48]:
df['key_word'] = df['key_word'].str.split(',').apply(lambda x: ', '.join(sorted(set(x), key=x.index)))

In [50]:
df['key_word'] =df['key_word'].str.replace(r'.','')

In [52]:
df['key_word'] =df['key_word'].str.replace(r':','')

In [54]:
df['key_word'] = df['key_word'].str.lstrip("', ")

In [56]:
df['key_word'] =df['key_word'].str.replace(r'(Note Notwithstanding and excessively is used here)','')

In [73]:
import pandas as pd
import nltk

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')
def extract_capitalized_tokens(sentence):
    # Tokenize the sentence
    tokens = nltk.word_tokenize(sentence)
    # Filter tokens that start with a capitalized first letter
    capitalized_tokens = [token for token in tokens if token[0].isupper()]
    return ', '.join(capitalized_tokens)
df['capitalized_tokens'] = df['key_word'].apply(extract_capitalized_tokens)

[nltk_data] Downloading package punkt to C:\Users\HP
[nltk_data]     Envy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [92]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r'Keyword','')

In [83]:
df['key_word'] = df['key_word'].str.lstrip(", ")

In [85]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r'None, ','')


In [87]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r' As, I, , ','')


In [88]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r'Here, We, , ','')


In [89]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r', Tax, A, , Technology, The, LNP\\','')


In [90]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r' Brain, Drain,','')


In [91]:
df['capitalized_tokens'] = df['capitalized_tokens'].str.replace(r' No, Economy, Yes, Tax, No, Investment, Yes, Technology, Yes, Environment, Yes, Cost, Yes','')


In [95]:
df.drop(['key_word'], axis=1, inplace=True)

In [96]:
df

,id,country,energy_source,comment,sentiment_score,capitalized_tokens
0,61,Australia,FMAA,https://12ft.io/https://www.smh.com.au/politic...,1.5632,"Policy, Jobs"
1,62,Australia,FMAA,The Coalition has attacked the production cred...,1.8852,"Tax, Investment, Environment, Cost"
2,63,Australia,FMAA,Production credits are better than handouts - ...,4.5928,"Jobs, Cost"
3,64,Australia,FMAA,"Roll the dice with our labour rates, our envir...",3.0,"Market, Risk, Investment, Strategy, Cost, Mana..."
4,65,Australia,FMAA,"Aiming for more economic complexity is good, b...",3.807,"Policy, Economy, Cost, Investment, Technology"
...,...,...,...,...,...,...
3771,3832,Australia,Other,Despite calls for no new #coal plants in order...,1,"Tax, Investment, Environment"
3772,3833,Australia,Other,Curious crossover: a political funding group i...,2,"Policy, Tax"
3773,3834,Australia,Other,Momentum is building for stronger action on cl...,4,"Environment, Climate, Change"
3774,3835,Australia,Nuclear Energy,Australia politics live: anti-trans protests ‘...,1,"Policy, Jobs, Economy, Note, There"


In [100]:
df['energy_source'].unique()

array(['FMAA', 'HHS', 'hydro', 'nuclearpower', 'RET', 'solarpower', 'SSP',
       'subsidies', 'windpower', 'Solar Energy', 'Other', 'Wind Energy',
       'Nuclear Energy', 'Bioenergy', 'Hydro Power', 'Geothermal'],
      dtype=object)

In [101]:
df['energy_source'] = df['energy_source'].apply(lambda x: "Hydro Power" if x =="hydro" else "Solar Energy" if x == "solarpower" else "Wind Energy" if x == "windpower" else "Nuclear Energy" if x == "nuclearpower" else x)

In [102]:
df['energy_source'].unique()

array(['FMAA', 'HHS', 'Hydro Power', 'Nuclear Energy', 'RET',
       'Solar Energy', 'SSP', 'subsidies', 'Wind Energy', 'Other',
       'Bioenergy', 'Geothermal'], dtype=object)

In [104]:
for i in range(len(df)):
    country = df['country'][i]
    energy_source = df['energy_source'][i]
    comment = df['comment'][i]
    key_word = df['capitalized_tokens'][i]
    sentiment_score = df['sentiment_score'][i]
    query="Insert into public_sentiment (country, energy_source, comment, key_word, sentiment_score) values (%s, %s, %s, %s, %s)"
    cursor.execute(query, (country, energy_source, comment, key_word, sentiment_score))
    connection.commit()
cursor.close()
connection.close()

In [105]:
df.to_csv(r"C:\Users\HP Envy\OneDrive - Swinburne University\Data Science\Innovation project\output_data\public_sentiment.csv", index = False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\HP Envy\OneDrive - Swinburne University\Data Science\Innovation project\output_data\public_sentiment.csv")

In [3]:
df

,id,country,energy_source,comment,sentiment_score,capitalized_tokens
0,61,Australia,FMAA,https://12ft.io/https://www.smh.com.au/politic...,1.5632,"Policy, Jobs"
1,62,Australia,FMAA,The Coalition has attacked the production cred...,1.8852,"Tax, Investment, Environment, Cost"
2,63,Australia,FMAA,Production credits are better than handouts - ...,4.5928,"Jobs, Cost"
3,64,Australia,FMAA,"Roll the dice with our labour rates, our envir...",3.0000,"Market, Risk, Investment, Strategy, Cost, Mana..."
4,65,Australia,FMAA,"Aiming for more economic complexity is good, b...",3.8070,"Policy, Economy, Cost, Investment, Technology"
...,...,...,...,...,...,...
3771,3832,Australia,Other,Despite calls for no new #coal plants in order...,1.0000,"Tax, Investment, Environment"
3772,3833,Australia,Other,Curious crossover: a political funding group i...,2.0000,"Policy, Tax"
3773,3834,Australia,Other,Momentum is building for stronger action on cl...,4.0000,"Environment, Climate, Change"
3774,3835,Australia,Nuclear Energy,Australia politics live: anti-trans protests ‘...,1.0000,"Policy, Jobs, Economy, Note, There"


In [4]:
def rename_capitalized_tokens(text):
    if "Infrastructure" in text:
        return "Infrastructure"
    elif "Cost" in text:
        return "Cost"
    elif "Tax" in text:
        return "Tax"
    elif "Technology" in text:
        return "Technology"
    elif "Investment" in text:
        return "Investment"
    elif "Environment" in text:
        return "Environment"
    elif "Economy" in text:
        return "Economy"
    elif "Jobs" in text:
        return "Jobs"
    elif "Innovation" in text:
        return "Innovation"
    elif "Energy" in text:
        return "Energy"
    else:
        return "Policy"

In [22]:
df['capitalized_tokens'] = df['capitalized_tokens'].fillna("Others")

In [23]:
df[df['comment']=="I can almost see Twiggy Forrest's enormous erection from here."]

,id,country,energy_source,comment,sentiment_score,capitalized_tokens
163,224,Australia,Hydro Power,I can almost see Twiggy Forrest's enormous ere...,3.0,Others


In [24]:
df['capitalized_tokens'] = df['capitalized_tokens'].apply(rename_capitalized_tokens)

In [25]:
df['capitalized_tokens']

0                 Jobs
1                 Cost
2                 Cost
3       Infrastructure
4                 Cost
             ...      
3771               Tax
3772               Tax
3773       Environment
3774           Economy
3775              Cost
Name: capitalized_tokens, Length: 3776, dtype: object

In [26]:
df.to_csv(r"C:\Users\HP Envy\OneDrive - Swinburne University\Data Science\Innovation project\output_data\public_sentiment(1).csv", index = False)

In [27]:
df =pd.read_csv(r"C:\Users\HP Envy\OneDrive - Swinburne University\Data Science\Innovation project\df.csv")

In [28]:
df

,index,file_name,text,policy,category,cleaned_category,numeric_category
0,0,ret_australia(5).pdf,Renewable Energy 67 (2014) 128e135\n\nContents...,ret,"ChatCompletionMessage(content=""I'm unable to f...",NaN,NaN
1,0,ret_australia.pdf,Energy Policy 71 (2014) 40–51\n\nContents list...,ret,"ChatCompletionMessage(content=""I couldn't find...",NaN,NaN
2,0,hydrogen headstart to power new jobs & industr...,Search\n\n\n\nHome (/) / Ministers (/minis...,hydrogen headstart,"ChatCompletionMessage(content='$2 billion', re...",$2 billion,2.0
3,0,ret_australia(4).pdf,Australian Journal of Agricultural and Resourc...,ret,"ChatCompletionMessage(content=""I couldn't find...",NaN,NaN
4,0,ret_australia(3).pdf,Energy Policy 62 (2013) 386–400\n\nContents li...,ret,"ChatCompletionMessage(content=""I cannot provid...",NaN,NaN
5,0,ret_australia(2).pdf,Renewable Energy 44 (2012) 119e127\n\nContents...,ret,"ChatCompletionMessage(content=""I couldn't find...",NaN,NaN
6,0,applications open for australia's solar sunsho..., Blog\n\n\n\n\nGeneral Solar & Storage News...,solar sunshot,ChatCompletionMessage(content='1 billion \n500...,1 billion \n500 million,1500.0
7,0,hydrogenheadstart.pdf,ReviewNot peer-reviewed versionHydrogen and Ot...,other,"ChatCompletionMessage(content=""I couldn't find...",NaN,NaN
8,0,future made in aus_2.pdf,A Future Made in Australia \n\nThis Budget inv...,future made,"ChatCompletionMessage(content='$22.7 billion',...",$22.7 billion,22.7
9,0,arena announces $1b solar sunshot program - en...,ARENA announces $1B Solar Sunshot program\n\nS...,solar sunshot,"ChatCompletionMessage(content='$10 million', r...",$10 million,10.0
